mnist training and testing
To create an mnist network, run the command "x = " and the result of (generateRandomNet ((784,50,50,10), ("Regular","Regular","Regular"), ("LeakyRelu","LeakyRelu","Sig"))) in haskell
    other networks would probably work, but this is the network I tested it on

Train the network by running the train executable (make sure the network file is saved as network.txt and the inputs are named trainingInputVector.txt and the correct output is named trainingOutputVector.txt)

you can open train.hs to adjust the training parameters and recompile it

To test the network, run the testing executable (with the same network file/name and testingInputVector.txt and testingInputVector.txt saved in the same directory)

To generate a random network, run generateRandomNet and then run the generateNetwork executable (generateRandomNet will output "layerTypes.txt", "neuronTypes.txt", "weights.txt", and "biases.txt" files, and the executable will use those files and output a "network.txt" file

To create testing/training input/output vector files, use: 
    toFile ("fileName" mnistDataString ((true if you want the inputs or false if you want the outputs), index where 
    you want to start taking the data from, number of data points to take)
the index is used to separate training and testing data, for example if you wanted to train on the first 500 data points and test on the next 500, the index used for the training data would be 0 and for the testing data would be 500.

**if you decide to recompile any of the files, use: "ghc filename -threaded"
to recompile the library (NeuralNet.hs) use: "ghc --make NeuralNet.hs  -threaded" (you should not have to do this)
when running the excutables use ./filename +RTS -N(number of logical cores your computer has) -RTS
(e.g. ./train +RTS -N4 -RTS)

In [1]:
import os.path
import numpy as np
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets("MNIST_data/", one_hot=True)
#subprocess library
#interprocess communication tends to be pretty slow

/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/importlib/_bootstrap.py:205: RuntimeWarning: compiletime version 3.5 of module 'tensorflow.python.framework.fast_tensor_util' does not match runtime version 3.6
  return f(*args, **kwds)


Extracting MNIST_data/train-images-idx3-ubyte.gz
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz


In [10]:
generateRandomNet ([784,784,392,392,196,196,98,98,10], ["Regular","Regular","Regular","Regular","Regular","Regular","Regular","Regular","Regular"],["Relu","Relu","Relu","Relu","Relu","Relu","Relu","Relu","Relu"])

1
2
3
4
5
6
7


In [7]:
#for layer sizes, you have (the amount of inputs, the amount of neurons in the first layer, the amount of neurons in the second layer, ..., the amount of outputs
#layerTypes is the type of each layer "Convolutional", "Recurrent", "Regular" (Feed forward)
    #currently only "Regular" layers are supported
#layerNeuronTypes is just the types of the neurons which compose a layer, "Sig","Per"(step function), "Relu", "NeuSoftMax", "Elu", "LeakyRelu", "Htan"
def generateRandomNet (layerSizes, layerTypes, layerNeuronTypes):
    layerTypesString = "["
    for i in layerTypes:
        layerTypesString = layerTypesString + '"' + i + '",'
    layerTypesString = (layerTypesString[:-1])+"]"
    neuronTypesString = "["
    print(1);
    for i in range (1,len(layerSizes)):
        neuronTypesString = neuronTypesString + "["
        for t in range (0, layerSizes[i]):
            neuronTypesString = neuronTypesString + '"' + layerNeuronTypes[i-1] + '",'
        neuronTypesString = (neuronTypesString[:-1])+"],"
    neuronTypesString = (neuronTypesString[:-1])+"]"
    weightsStrings = ["["]
    print(2);
    for i in range (0, (len(layerSizes)-1)):
        weightsStrings.append("[")
        for t in range (0, layerSizes[i+1]):
            weightsStrings.append("[")
            for x in range (0, layerSizes[i]):
                weightsStrings.append(''+(str((np.random.rand(1)[0])-.5)) + ",")
            localThing = weightsStrings.pop()
            weightsStrings.append((localThing[:-1])+"],")
        localThing = weightsStrings.pop()
        weightsStrings.append((localThing[:-1])+"],")
    localThing = weightsStrings.pop()
    weightsStrings.append((localThing[:-1])+"]")
    weightsString = ''.join([i for i in weightsStrings])
    biasesStrings = ["["]
    print(3);
    for i in range (1, len(layerSizes)):
        biasesStrings.append("[")
        for t in range (0, layerSizes[i]):
            biasesStrings.append(''+(str((np.random.rand(1)[0])-.5))+ ',')
        localThing = biasesStrings.pop()
        biasesStrings.append((localThing[:-1])+"],")
    localThing = biasesStrings.pop()
    biasesStrings.append((localThing[:-1])+"]")
    biasesString = ''.join([i for i in weightsStrings])
    print(4);
    toFile ("layerTypes.txt", layerTypesString)
    print(5);
    toFile ("neuronTypes.txt", neuronTypesString)
    print(6);
    toFile ("weights.txt", weightsString)
    print(7);
    toFile ("biases.txt", biasesString)

    
    
#:: [String] -> [[String]] -> [[[Double]]] -> [[Double]]
#takes in a list of layer types in string form, a 2d list of all of the neuron types in string form, a 3d list of all of the weights, and a 2d list of all of the Biases and returns a list of layers (or a network)

In [3]:
#images should be true if you want the images (inputs) and false if you want the numbers (outputs)
#startpoint is the index of the dataset from which it starts taking data, you should start the testing/ training data at different points
#number is the number of data points to take
def mnistDataString (images, startPoint, number):
    command = "["
    if images:
        for i in range (startPoint,(startPoint + number)):
            command = command + "["
            for t in range (0,784):
                command = command + (str (mnist.train.images[i][t])) +","
            command = (command[:-1])+"],"
    else:
        for i in range (startPoint,(startPoint + number)):
            command = command + "["
            for t in range (0,10):
                command = command + (str (mnist.train.labels[i][t])) +","
            command = (command[:-1])+"],"
    command = (command[:-1])+"]"
    return (command)

In [4]:
def mnistTestOLD (number):
    command = "testClassification y ["
    for t in range (0,number):
        command = command + "["
        for i in range (0,784):
            command = command + (str(mnist.test.images[t][i])) + ","
        command = (command[:-1])+"],"
    command = (command[:-1])+"] ["
    for t in range (0,number):
        command = command + "["
        for i in range (0,10):
            command = command + (str(mnist.test.labels[t][i])) + ","
        command = (command[:-1])+"],"
    command = (command[:-1])+"]"
    return (command)

In [5]:
def mnistTrainOLD (epochs, splits, dataPoints, stepSize):
    command = "train " + (str (epochs)) + " " + (str (splits))+ " x " + "["
    for i in range (0,dataPoints):
        command = command + "["
        for t in range (0,784):
            command = command + (str (mnist.train.images[i][t])) +","
        command = command = (command[:-1])+"],"
    command = command = (command[:-1])+"] ["
    for i in range (0,dataPoints):
        command = command + "["
        for t in range (0,10):
            command = command + (str (mnist.train.labels[i][t])) +","
        command = (command[:-1])+"],"
    command = (command[:-1])+"] "+ (str (stepSize))
    return (command)

In [9]:
def toFile (fileName, inpu):
    fh = open(fileName,"w")
    fh.write(inpu)
    fh.close()
